In [1]:
import numpy as np
import pandas as pd

In [140]:
import string
import math

def text_match(input_text,database_text,questions_database):
    input_text = input_text.lower()
    punc = string.punctuation
    for i in punc:
        input_text = input_text.replace(i,' ')
    words_input = input_text.split()
    words_database = database_text.split()
    words = set(words_input).union(set(words_database))
    words_dict = dict()
    i = 0
    for word in words:
        words_dict[word] = i
        i += 1
        
    # Generate the word frequency vector
    input_vector = [0]*len(words_dict)
    for word in words_input:
        input_vector[words_dict[word]]+=1            
    database_vector = [0]*len(words_dict)
    for word in words_database:
        database_vector[words_dict[word]]+=1
    
    # Calculate the IDF
    question_list = questions_database['question'].tolist()
    total = len(question_list) # number of sentences in the database
    IDF_vector = [1]*len(words_dict)
    for key in words_dict:
        i = 0
        for item in question_list:
            if key in item:
                i+=1
        IDF_vector[words_dict[key]] = math.log(total/(i+1))
        
    # Calculate the TF-IDF value
    input_array = np.array(input_vector)
    database_array = np.array(database_vector)
    IDE_array = np.array(IDF_vector)
    input_vector = (input_array*IDE_array).tolist()
    database_vector = (database_array*IDE_array).tolist()
     
    # Calculate the cos value
    sum = 0
    sq1 = 0
    sq2 = 0
    for i in range(len(input_vector)):
        sum += input_vector[i] * database_vector[i]
        sq1 += pow(input_vector[i],2)
        sq2 += pow(database_vector[i],2)
    try: 
        result = round(float(sum) / (math.sqrt(sq1) * math.sqrt(sq2)),3)
    except ZeroDivisionError:
        result = 0.0
    
    return result

# find the most mateched question class
def find_most_match(input_text,questions_database):
    questions_database['score'] = 0
    for i in range(len(questions_database)):
        questions_database.loc[i,'score'] = text_match(input_text,questions_database.loc[i,'question'],questions_database)
#     score_mean = questions_database.groupby('class')['score'].mean()
#     question_class = score_mean.idxmax()
    questions_database = questions_database.sort_values(by = 'score', ascending = False)
    if questions_database.loc[questions_database['score'].idxmax(),'score'] < 0.5:
        return False
    else:
        questions_class = questions_database.loc[questions_database['score'].idxmax(),'class']
        return questions_class

def find_answer(input_city,answers_database,question_class):
    while not input_city.lower() in answers_database.loc[:,'city'].tolist():
        input_city = input('Sorry, I cannot find the city. Please type again. Thanks ')
    answers = answers_database.loc[answers_database['city']==input_city.lower()][question_class].tolist()[0]
    return answers

In [146]:
def dialogue(questions,answers):
    print('Hello! Nice to see you!')
    question_class = True
    input_city = False
    while question_class:
        input_text = input('Just ask me questions, and I will try to help you! ')
        while not find_most_match(input_text,questions):
            input_text = input('Sorry, I cannot help you with this question at the moment. Do you mind changing another one? ')    
        question_class = find_most_match(input_text,questions)
        if question_class == 'ending':
            print(input_text)
            return False
        elif question_class == 'thank':
            print('Welcome!')
        else:
            if not input_city:
                input_city = input('OK. In/To which city upon your question? ')
                answer = find_answer(input_city,answers,question_class)
                print('The %s is %s'%(question_class,answer))
            else: 
                input_city_confirm = input('Is it the same city %s '%(input_city)+'as you have mentioned? ' )
                if input_city_confirm.lower() == 'yes':
                    answer = find_answer(input_city,answers,question_class)
                    print('The %s is %s'%(question_class,answer))
                else:
                    input_city = input('OK. In/To which city upon your question? ')
                    answer = find_answer(input_city,answers,question_class)
                    print('The %s is %s'%(question_class,answer))

In [148]:
questions = pd.read_csv('questions.csv')
answers = pd.read_csv('answers.csv')
dialogue(questions,answers)

Hello! Nice to see you!
Just ask me questions, and I will try to help you! What is the weather like today?
OK. In/To which city upon your question? Gothenburg
The weather is sunny, 10 degrees celsius 
Just ask me questions, and I will try to help you! Could you recommend a restaurant?
Is it the same city Gothenburg as you have mentioned? Yes
The restaurant is SWEDISH TASTE
Just ask me questions, and I will try to help you! What time is the next bus?
Is it the same city Gothenburg as you have mentioned? No
OK. In/To which city upon your question? Stockholm
The bus is 10 min later at B bus station 
Just ask me questions, and I will try to help you! Where is the nearest hospital?
Sorry, I cannot help you with this question at the moment. Do you mind changing another one? No, thank you
Welcome!
Just ask me questions, and I will try to help you! See you
See you


False